#NOME FIGO
Art and history of art are no sealed compartments: they are heavily inter-dependent with social, political, economic factors, which in turn influence our very perception of what art is.

Cultural institutions and museums in particular play a fundamental role in this intertwined dynamics: through their selection activity, they have the potential to shape the public understanding of arts and its modifications throughout time. 
In some way, what makes into museums makes into history of art.

From these considerations stems our analysis: how do external (social, political, economic) factors influence the perception of art and its history?
A way to investigate it is by looking at the greatest and most representative museums around the world, and at their acquisition policies and campaigns in particular.

Our key questions:
In which ways have the acquisition campaigns of the major museums in the world changed throughout the years? 


Our workflow:
1. Interrogate WikiData:
    - What are the biggest collections around the world?
2. Find csv files for some of the major museums.
3. Select some representative time slots (both internal and external factors).
4. Analyse acquisitions during these time slots for every museum and compare:
    a) Difference between different slots in the same museum;
    b) Difference between different museums for the same time slot;

Our questions:
- What was the initial nucleus of each museum? 
- Internal survey: Is there a significant date or decade for the acquisitions? 
- External survey: What are the acquisition trends around the Xs/between the x and the y? / What are the acquisition trends within and across these museums? 
- During these years, who are the most represented makers? What is the most represented gender? What is the most represented movement? What is the most represented nationality? 


We analysed 5/4 of the (MET, MoMa, N+, Cleveland?, Tate) 

Wikidata interrogation: failure.

1. What are the largest art collections?

SELECT ?museum (COUNT(?work) AS ?works) WHERE {
  ?work wdt:P195 ?museum.
  ?museum wdt:P31 wd:Q207694
  }
GROUP BY ?museum 
ORDER BY DESC(?works)

2.  Which were the most visited museums in 2018?

SELECT ?museumLabel ?visitors ?year
WHERE {
  ?museum wdt:P31 wd:Q207694;
          wdt:P1705 ?museumLabel;
          wdt:P1174 ?visitors;
          p:P1174/pq:P585 ?year .
FILTER(YEAR(?year) = 2018).
}

ORDER BY DESC(?visitors)

Since WikiData was not providing reliable results, we decided to go back to its sources (The Art Newspaper https://www.theartnewspaper.com/) and manually collect data about the most visited museums in the last four years(2018-2022).

https://onedrive.live.com/view.aspx?resid=E34DDE1A3F2F2160!138&ithint=file%2cxlsx&authkey=!AN4u-K4bko37iOU
    
We verified the availability of open datasets for each of the top 20 most visited museums on this GitHub repository (https://github.com/Ambrosiani/museums-on-github), containing a list of museums with GitHub accounts.

Our analysis led us to the decision to focus on four museums:
- Tate Modern, London
- MoMa, NY
- Met, NY
- National Gallery of Art, Washington DC

**Info generale sui musei.

In [1]:
import pandas as pd
import csv
import re

In [2]:
spreadsheet = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')
pd.set_option('display.max_columns', None)
artworks = spreadsheet[['Title', 'Artist', 'ConstituentID', 'Nationality', 'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'CreditLine', 'Classification', 'Department', 'DateAcquired', 'URL']]
artists = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artists.csv')
artists["ConstituentID"] = artists["ConstituentID"].astype(str)
MoMa = pd.merge(artworks,artists[['ConstituentID', 'Wiki QID']],on='ConstituentID', how='left')
MoMa.rename(columns = {'ConstituentID':'Id', 'BeginDate':'BirthDate', 'EndDate':'DeathDate'}, inplace = True)
MoMa.Date = MoMa.Date.fillna('Not available')
MoMa['Date'] = MoMa['Date'].astype(str)
#MoMa.to_csv("MoMa.csv")
MoMa.head(30)

,Title,Artist,Id,Nationality,BirthDate,DeathDate,Gender,Date,Medium,CreditLine,Classification,Department,DateAcquired,URL,Wiki QID
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,Fractional and promised gift of Jo Carole and ...,Architecture,Architecture & Design,1996-04-09,http://www.moma.org/collection/works/2,Q84287
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,(French),(1944),(0),(Male),1987,Paint and colored pencil on print,Gift of the architect in honor of Lily Auchinc...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/3,Q312838
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,(Austrian),(1876),(1957),(Male),1903,"Graphite, pen, color pencil, ink, and gouache ...",Gift of Jo Carole and Ronald S. Lauder,Architecture,Architecture & Design,1997-01-15,http://www.moma.org/collection/works/4,Q1336246
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,(),(1944),(0),(Male),1980,Photographic reproduction with colored synthet...,Purchase and partial gift of the architect in ...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/5,Q123966
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,(Austrian),(1876),(1957),(Male),1903,"Graphite, color pencil, ink, and gouache on tr...",Gift of Jo Carole and Ronald S. Lauder,Architecture,Architecture & Design,1997-01-15,http://www.moma.org/collection/works/6,Q1336246
5,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,(),(1944),(0),(Male),1976-77,Gelatin silver photograph,Purchase and partial gift of the architect in ...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/7,Q123966
6,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,(),(1944),(0),(Male),1976-77,Gelatin silver photographs,Purchase and partial gift of the architect in ...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/8,Q123966
7,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,(),(1944),(0),(Male),1976-77,Gelatin silver photograph,Purchase and partial gift of the architect in ...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/9,Q123966
8,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,(),(1944),(0),(Male),1976-77,Gelatin silver photograph,Purchase and partial gift of the architect in ...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/10,Q123966
9,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,(),(1944),(0),(Male),1976-77,Gelatin silver photograph,Purchase and partial gift of the architect in ...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/11,Q123966


In [3]:
spreadsheet = pd.read_csv('https://raw.githubusercontent.com/tategallery/collection/master/artwork_data.csv')
pd.set_option('display.max_columns', None)
artworks = spreadsheet[['artist', 'artistId', 'title', 'medium', 'creditLine', 'year', 'acquisitionYear', 'url']]
artworks.rename(columns = {'artistId':'id'}, inplace = True)
artworks.id = artworks.id.astype(str)
artists = pd.read_csv('https://raw.githubusercontent.com/tategallery/collection/master/artist_data.csv')
artists["id"] = artists["id"].astype(str)
Tate = pd.merge(artworks,artists[['id', 'gender', 'yearOfBirth', 'yearOfDeath']],on='id', how='left')
Tate.rename(columns = {'artist':'Artist', 'id':'Id', 'title':'Title', 'yearOfBirth':'BirthDate', 'yearOfDeath':'DeathDate', 'medium':'Medium', 'creditLine':'CreditLine', 'year':'Date', 'acquisitionYear':'DateAcquired', 'url':'URL', 'gender':'Gender'}, inplace = True)
#Tate.to_csv("Tate.csv")
Tate.head(3)

C:\Users\Francesca\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Francesca\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\Francesca\AppData\Roaming\Python\Python39\site-packages\pandas\core\generic.py:5489: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

,Artist,Id,Title,Medium,CreditLine,Date,DateAcquired,URL,Gender,BirthDate,DeathDate
0,"Blake, Robert",38,A Figure Bowing before a Seated Old Man with h...,"Watercolour, ink, chalk and graphite on paper....",Presented by Mrs John Richmond 1922,NaN,1922.0,http://www.tate.org.uk/art/artworks/blake-a-fi...,Male,1762.0,1787.0
1,"Blake, Robert",38,"Two Drawings of Frightened Figures, Probably f...",Graphite on paper,Presented by Mrs John Richmond 1922,NaN,1922.0,http://www.tate.org.uk/art/artworks/blake-two-...,Male,1762.0,1787.0
2,"Blake, Robert",38,The Preaching of Warning. Verso: An Old Man En...,Graphite on paper. Verso: graphite on paper,Presented by Mrs John Richmond 1922,1785.0,1922.0,http://www.tate.org.uk/art/artworks/blake-the-...,Male,1762.0,1787.0


In [4]:
spreadsheet = pd.read_csv('https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv')
pd.set_option('display.max_columns', None)
Met = spreadsheet[['AccessionYear', 'Title', 'Culture', 'Artist Display Name', 'Artist Nationality', 'Artist Begin Date', 'Artist End Date', 'Artist Gender', 'Artist Wikidata URL', 'Object End Date', 'Medium', 'Credit Line', 'Classification', 'Link Resource', 'Object Wikidata URL']]
Met.rename(columns = {'Artist Display Name':'Artist', 'id':'Id', 'Artist Begin Date':'BirthDate', 'Artist End Date':'DeathDate', 'Credit Line':'CreditLine', 'Object End Date':'Date', 'AccessionYear':'DateAcquired', 'Artist Wikidata URL':'Wiki QID', 'Artist Gender':'Gender', 'Link Resource':'URL', 'Artist Nationality':'Nationality'}, inplace = True)
#Met.to_csv("Met.csv")
Met.head(3)

C:\Users\Francesca\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (5,7,10,11,12,13,14,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Francesca\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,DateAcquired,Title,Culture,Artist,Nationality,BirthDate,DeathDate,Gender,Wiki QID,Date,Medium,CreditLine,Classification,URL,Object Wikidata URL
0,1979.0,One-dollar Liberty Head Coin,NaN,James Barton Longacre,American,1794,1869,NaN,https://www.wikidata.org/wiki/Q3806459,1853,Gold,"Gift of Heinz L. Stoppelmann, 1979",NaN,http://www.metmuseum.org/art/collection/search/1,NaN
1,1980.0,Ten-dollar Liberty Head Coin,NaN,Christian Gobrecht,,1785,1844,NaN,https://www.wikidata.org/wiki/Q5109648,1901,Gold,"Gift of Heinz L. Stoppelmann, 1980",NaN,http://www.metmuseum.org/art/collection/search/2,NaN
2,1967.0,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1927,Gold,"Gift of C. Ruxton Love Jr., 1967",NaN,http://www.metmuseum.org/art/collection/search/3,NaN


In [5]:
spreadsheet = pd.read_csv('https://raw.githubusercontent.com/NationalGalleryOfArt/opendata/main/data/objects.csv')
pd.set_option('display.max_columns', None)
Nga = spreadsheet[['accessionnum', 'title', 'endyear', 'medium', 'attribution', 'creditline', 'classification']]
Nga.rename(columns = {'attribution':'Artist', 'id':'Id', 'title':'Title', 'medium':'Medium', 'creditline':'CreditLine', 'endyear':'Date', 'accessionnum':'DateAcquired', 'classification':'Classification', 'Object End Date':'Date'}, inplace = True)
#Nga.to_csv("Nga.csv")
Nga.head(3)

C:\Users\Francesca\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3172: DtypeWarning: Columns (28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Francesca\AppData\Roaming\Python\Python39\site-packages\pandas\core\frame.py:4438: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,DateAcquired,Title,Date,Medium,Artist,CreditLine,Classification
0,1994.43.3,Costume de serge blanc plissé.,1919.0,offset lithograph with watercolor applied thro...,French 20th Century,George M. Richter Archive,Print
1,1994.43.4,"Robe de lainage, couleur taupe avec de la brod...",1919.0,offset lithograph with watercolor applied thro...,French 20th Century,George M. Richter Archive,Print
2,1994.59.11,Breton Village,1890.0,oil on canvas,Odilon Redon,Collection of Mr. and Mrs. Paul Mellon,Painting


Next, we decided to explore the four collections.
- How many items does each collection contain?
- Which timespan do items cover overall?
- First and last acquisition date for each museum. Tate's csv last update dates back to 2014.
- Total artists' number.
- Most represented artist, gender and nationality in general?
- Which artworks typologies are represented in each museum and in which proportion?

In [6]:
museums=[MoMa, Met, Tate, Nga]
names = ['Moma', 'Met', 'Tate', 'Nga']
for museum in museums:
    selected_rows = museum[~museum['Title'].isnull()]
    name = names.pop(0)
    print("Total items at", name, ":", len(selected_rows.index))

Total items at Moma : 139912
Total items at Met : 448619
Total items at Tate : 69201
Total items at Nga : 137923


In [7]:
def cleanDates(date):
    if '-' in date:
        splitted = date.split('-')
        year = splitted[0]
        return year
    if '/' in date:
        splitted = date.split('/')
        year = splitted[0]
        return year
    if ',' in date:
        splitted = date.split(',')
        year = splitted[0]
        return year
    if '.' in date:
        splitted = date.split('.')
        year = splitted[0]
        return year
    else:
        return date

In [8]:
def cleanDates(date):
    date = re.sub('', '', date) 

    return date

In [9]:
def cleanDates(date):
    if '-' in date:
        splitted = date.split('-')
        date = ' '.join(splitted) 
    if '/' in date:
        splitted = date.split('/')
        date = ' '.join(splitted) 
    if ',' in date:
        splitted = date.split(',')
        date = ' '.join(splitted) 
    if '.' in date:
        splitted = date.split('.')
        date = ' '.join(splitted) 
        
    x = re.search("\d{4}", date)
    if x:
        date = x.group()
    return date

In [10]:
MoMaNew = MoMa.copy(deep=True)
MoMaNew["Date"] = MoMaNew["Date"].apply(cleanDates)
MoMaNew.head(20)

,Title,Artist,Id,Nationality,BirthDate,DeathDate,Gender,Date,Medium,CreditLine,Classification,Department,DateAcquired,URL,Wiki QID
0,"Ferdinandsbrücke Project, Vienna, Austria (Ele...",Otto Wagner,6210,(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,Fractional and promised gift of Jo Carole and ...,Architecture,Architecture & Design,1996-04-09,http://www.moma.org/collection/works/2,Q84287
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,(French),(1944),(0),(Male),1987,Paint and colored pencil on print,Gift of the architect in honor of Lily Auchinc...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/3,Q312838
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,(Austrian),(1876),(1957),(Male),1903,"Graphite, pen, color pencil, ink, and gouache ...",Gift of Jo Carole and Ronald S. Lauder,Architecture,Architecture & Design,1997-01-15,http://www.moma.org/collection/works/4,Q1336246
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,(),(1944),(0),(Male),1980,Photographic reproduction with colored synthet...,Purchase and partial gift of the architect in ...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/5,Q123966
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,(Austrian),(1876),(1957),(Male),1903,"Graphite, color pencil, ink, and gouache on tr...",Gift of Jo Carole and Ronald S. Lauder,Architecture,Architecture & Design,1997-01-15,http://www.moma.org/collection/works/6,Q1336246
5,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,(),(1944),(0),(Male),1976,Gelatin silver photograph,Purchase and partial gift of the architect in ...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/7,Q123966
6,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,(),(1944),(0),(Male),1976,Gelatin silver photographs,Purchase and partial gift of the architect in ...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/8,Q123966
7,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,(),(1944),(0),(Male),1976,Gelatin silver photograph,Purchase and partial gift of the architect in ...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/9,Q123966
8,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,(),(1944),(0),(Male),1976,Gelatin silver photograph,Purchase and partial gift of the architect in ...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/10,Q123966
9,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,(),(1944),(0),(Male),1976,Gelatin silver photograph,Purchase and partial gift of the architect in ...,Architecture,Architecture & Design,1995-01-17,http://www.moma.org/collection/works/11,Q123966


1. What are the most acquired artists in museums (in general)?
    - is there a gender gap in the selection of artists?
    - what are the most represented nationalities (in general)?
    - what are the most represented movements or genres (in general)?
2. How have acquisition criteria changed (over time) in museums?
    - In which years are artists' works mostly acquired?
    - when does the gender gap decreases (if it does)?
    - in which years artists' nationalities more influent on the selection?
    - in which years artists'movements/genres more influent on the selection?
3. if we compare criteria of all museums, in general and over time, do we see any similarity or significant difference?
    - do certain museums acquire more works based on artists/artists' gender/nationality/movement than others?

Acquisition criteria.
1.  In which years are artists' works mostly acquired?<br>
To answer, we need to count how many times each year shows up in the DateAcquired column.

In [82]:
MoMa['year'] = pd.DatetimeIndex(MoMa['DateAcquired']).year
MoMaNew['DateAcquired'] = MoMa['year']
MoMaNew = MoMaNew[MoMaNew['DateAcquired'].notna()]
MoMaNew.to_csv("MoMaNew.csv")

In [108]:
with open('MoMaNew.csv', mode='r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    years={}
    for item in reader:
        if item['DateAcquired']not in years:
            years[item['DateAcquired']]= 1
        else:
            years[item['DateAcquired']]+= 1

    print(years)
    #all_years=list(years.keys())
    #print(sorted(all_years))
    

{'1996.0': 972, '1995.0': 968, '1997.0': 950, '1966.0': 1851, '1980.0': 672, '2000.0': 1886, '1990.0': 1268, '1989.0': 890, '1992.0': 1194, '1988.0': 1019, '1993.0': 1243, '1947.0': 448, '1991.0': 822, '1998.0': 1278, '1973.0': 1089, '1987.0': 1317, '1978.0': 1348, '1981.0': 1347, '1975.0': 981, '1954.0': 881, '1986.0': 1024, '1979.0': 1160, '1999.0': 1807, '1965.0': 2018, '1984.0': 870, '1967.0': 2411, '1994.0': 944, '1969.0': 2194, '1985.0': 1045, '1964.0': 12828, '1982.0': 851, '1976.0': 902, '1972.0': 862, '1974.0': 3371, '1983.0': 1194, '1963.0': 951, '1968.0': 6894, '2001.0': 4170, '1977.0': 1208, '1939.0': 399, '1949.0': 1582, '2002.0': 635, '1948.0': 421, '1943.0': 1131, '1946.0': 171, '1958.0': 703, '1957.0': 305, '1960.0': 450, '1952.0': 548, '1934.0': 256, '1945.0': 290, '1955.0': 443, '1950.0': 453, '1956.0': 1127, '1953.0': 474, '1944.0': 345, '1962.0': 565, '1961.0': 553, '1942.0': 933, '1938.0': 124, '1951.0': 631, '1970.0': 1234, '1971.0': 1041, '1959.0': 831, '1941.0':

Visualizzazione :)

In [109]:
new_dict={}
for key in years:
    key_int=key.split('.')[0]
    key_int=int(key_int)
    if key_int in range(1928,1941):
        if '1930s' not in new_dict.keys():
               new_dict['1930s']= years[key]
        else:
            new_dict['1930s'] += years[key]
    if key_int in range(1940,1951):
        if '1940s' not in new_dict.keys():
               new_dict['1940s']= years[key]
        else:
            new_dict['1940s'] += years[key]
    
    if key_int in range(1950,1961):
        if '1950s' not in new_dict.keys():
               new_dict['1950s']= years[key]
        else:
            new_dict['1950s'] += years[key]
    
    if key_int in range(1960,1971):
        if '1960s' not in new_dict.keys():
               new_dict['1960s']= years[key]
        else:
            new_dict['1960s'] += years[key]
    
    if key_int in range(1970,1981):
        if '1970s' not in new_dict.keys():
               new_dict['1970s']= years[key]
        else:
            new_dict['1970s'] += years[key]
    if key_int in range(1980,1991):
        if '1980s' not in new_dict.keys():
               new_dict['1980s']= years[key]
        else:
            new_dict['1980s'] += years[key]
    
    if key_int in range(1990,2001):
        if '1990s' not in new_dict.keys():
               new_dict['1990s']= years[key]
        else:
            new_dict['1990s'] += years[key]
        
    
print(new_dict)
    

{'1990s': 13332, '1960s': 31949, '1970s': 13868, '1980s': 11497, '1940s': 8274, '1950s': 6846, '1930s': 3318}
